<a href="https://colab.research.google.com/github/azilya/torch_tutorials/blob/main/lstm_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from string import punctuation

import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [ ]:
punctuation += "«»„“”‛’—–‒"
RE_PUNCT = re.compile(f"[{re.escape(punctuation)}]")


def clean(_str):
    _str0 = re.sub(RE_PUNCT, "", _str)
    res = re.sub(r"\s+", " ", _str0).lower()
    return res.strip() + "."


sentences = []
with open("train_0.txt") as corpus:
    for line in tqdm(corpus):
        if len(line.strip()) > 0:
            sentences.append(clean(line))

all_chars = sorted(list(set("".join(sentences))))
seq_len = 100

11326it [00:00, 36601.30it/s]


In [ ]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)

class CharModel(nn.Module):
    def __init__(self, vocab_size, emb_size, hid_size=256) -> None:
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(
            emb_size,
            hid_size,
            num_layers=2,
            batch_first=True,
            dropout=0.2,
            bidirectional=True,
        )
        self.dropout = nn.Dropout(0.2)
        self.activation = nn.Softmax(dim=-1)
        self.linear = nn.Linear(hid_size * 2, vocab_size)

    def forward(self, x):
        x = self.emb(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear(self.activation(self.dropout(x)))
        return x


model = CharModel(vocab_size=len(all_chars), emb_size=seq_len)
model.cuda()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

In [ ]:
X_s = []
y_s = []
for sent in tqdm(sentences):
    for i in range(len(sent) - seq_len):
        seq = sent[i : i + seq_len]
        X = [all_chars.index(c) for c in seq]
        y = all_chars.index(sent[i + seq_len])
        X_s.append(X)
        y_s.append(y)


# X_s = np.array(X_s) / len(all_chars)
# y_s = np.array(y_s) / len(all_chars)
dataset = TensorDataset(torch.tensor(X_s), torch.tensor(y_s))

train_set, test_set = train_test_split(dataset, test_size=0.15)

100%|██████████| 5663/5663 [02:32<00:00, 37.12it/s]


In [ ]:
n_epochs = 10
batch_size = 128
log_steps = 10000

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

In [ ]:
step = 0
for e in range(n_epochs):
  model.train()
  for batch in train_loader:
    inputs, truth = (t.to("cuda") for t in batch)
    pred = model(inputs)
    loss = loss_fn(pred, truth)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    step += 1
    if step % log_steps == 0:
      print(f"Step={step} {loss=:.4f}")
  model.eval()
  eval_loss = 0
  for batch in test_loader:
    inputs, truth = (t.to("cuda") for t in batch)
    with torch.no_grad():
      pred = model(inputs)
      loss = loss_fn(pred, truth)
    eval_loss += loss.detach().cpu().item()
  eval_loss /= len(test_loader)
  print(f"=== Epoch {e} eval loss: {eval_loss:.4f} ===")

Step=10000 loss=2.9970
=== Epoch 0 eval loss: 2.8283 ===
Step=20000 loss=2.4698
